In [7]:
pip install torch torchvision opencv-python pillow


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 192.2/192.2 MB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
pip install deeplake

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 568.4/568.4 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 82.1/82.1 kB 1.6 MB/s eta 0:00:00
     -------------------------------------- 133.3/133.3 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 56.8/56.8 kB 990.4 kB/s eta 0:00:00
     -------------------------------------- 115.3/115.3 kB 1.7 MB/s eta 0:00:00
  Created wheel for deeplake: filename=deeplake-3.8.1-py3-none-any.whl size=685277 sha256=288fad92b633ee20ac805b3a4636c696df8e06f796bbcdbbb6f5d6b32560adec
  Stored in directory: c:\users\91920

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [14]:
import deeplake
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
import torchvision


# Load the "AFW" dataset from Deeplake
ds = deeplake.load("hub://activeloop/AFW")

# Define a custom dataset class for the Deeplake dataset
class CustomFaceDataset(Dataset):
    def __init__(self, deeplake_dataset, transform=None):
        self.deeplake_dataset = deeplake_dataset
        self.transform = transform

    def __len__(self):
        return len(self.deeplake_dataset)

    def __getitem__(self, idx):
        image, target = self.deeplake_dataset[idx]
        image = Image.open(image)

        if self.transform:
            image = self.transform(image)

        return image, target

# Define transformations, such as resizing and normalizing the images
transform = T.Compose([
    T.Resize((300, 300)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create the custom dataset
custom_dataset = CustomFaceDataset(ds, transform=transform)

# Create a data loader
batch_size = 4  # Define your batch size
num_workers = 4  # Define the number of workers
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# Load a pre-trained Faster R-CNN model from torchvision
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Modify the model's classifier to match the number of classes in your dataset
num_classes = 2  # 1 for face, 1 for background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Set the model in training mode
model.train()

# Define a custom loss function for your face detection task (e.g., a combination of RPN and box regression loss)
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init()

    def forward(self, predictions, targets):
        rpn_objectness_loss = predictions['loss_objectness']
        rpn_box_reg_loss = predictions['loss_rpn_box_reg']
        classifier_loss = predictions['loss_classifier']
        box_reg_loss = predictions['loss_box_reg']

        # Define weight factors for different loss components
        weight_rpn_objectness = 1.0
        weight_rpn_box_reg = 1.0
        weight_classifier = 1.0
        weight_box_reg = 1.0

        loss_rpn_objectness = rpn_objectness_loss * weight_rpn_objectness
        loss_rpn_box_reg = rpn_box_reg_loss * weight_rpn_box_reg
        loss_classifier = classifier_loss * weight_classifier
        loss_box_reg = box_reg_loss * weight_box_reg

        total_loss = loss_rpn_objectness + loss_rpn_box_reg + loss_classifier + loss_box_reg

        return total_loss



|\

Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/AFW



\

hub://activeloop/AFW loaded successfully.



C:\Users\91920\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\91920\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\91920/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|████████████████████████████████████████████████████████████████████████████████| 160M/160M [04:11<00:00, 667kB/s]


AttributeError: 'super' object has no attribute '_CustomLoss__init'

In [16]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()



custom_loss = CustomLoss()

# Define an optimizer (e.g., Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define a learning rate scheduler (e.g., StepLR)
lr_scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

# Train the model on your dataset
num_epochs = 10  # Define the number of training epochs
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

for epoch in range(num_epochs):
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = custom_loss(loss_dict, targets)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

# Save the trained model
torch.save(model.state_dict(), 'face_detection_model.pth')


RuntimeError: DataLoader worker (pid(s) 22868, 19560, 24264, 24964) exited unexpectedly

In [ ]:

#use saved model
import torchvision.transforms as T

# Load the trained model
model.load_state_dict(torch.load('face_detection_model.pth'))
model.eval()  # Set the model to evaluation mode

# Define a list of class labels for face and background
class_labels = ['background', 'face']

# Load an image for face detection (you can replace 'image_path' with the path to your image)
image_path = 'path_to_your_image.jpg'
image = Image.open(image_path).convert("RGB")

# Define a transformation for the input image
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Apply the transformation to the image
image = transform(image)

# Make a prediction using the trained model
with torch.no_grad():
    prediction = model([image])

# Get the predicted bounding boxes, labels, and scores
boxes = prediction[0]['boxes']
labels = prediction[0]['labels']
scores = prediction[0]['scores']

# Filter the results to only include face detections
face_boxes = boxes[labels == 1]  # 1 corresponds to 'face' class
face_scores = scores[labels == 1]

# You can now use the 'face_boxes' and 'face_scores' to work with the detected faces.


In [ ]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection import maskrcnn_resnet50_fpn
import torch.nn as nn
from torch.optim import Adam


# Define paths to the dataset folders (JPG images and PTS annotations)
jpg_folder = 'C:/Users/ravis/Downloads/SHAREit/pc/file/AFW/jpg'
pts_folder = 'C:/Users/ravis/Downloads/SHAREit/pc/file/AFW/pts'

# Create lists of JPG and PTS file paths
jpg_files = [os.path.join(jpg_folder, filename) for filename in os.listdir(jpg_folder) if filename.endswith('.jpg')]
pts_files = [os.path.join(pts_folder, filename) for filename in os.listdir(pts_folder) if filename.endswith('.pts')]

# Print the list of JPG file paths
print("JPG Files:")
for jpg_file in jpg_files:
    print(jpg_file)

# Print the list of PTS file paths
print("\nPTS Files:")
for pts_file in pts_files:
    print(pts_file)

# Define a function to parse PTS annotation files and generate bounding box coordinates
def generate_bbox_from_pts(pts_file):
    with open(pts_file, 'r') as file:
        lines = file.readlines()

    # Initialize lists to store facial landmark points
    landmarks = []

    # Iterate through lines to extract facial landmark points
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 2:
            try:
                x, y = map(float, parts)
                landmarks.append((x, y))
            except ValueError:
                continue  # Skip lines that cannot be converted to float

    if len(landmarks) >= 2:
        # Calculate bounding box coordinates
        x_values, y_values = zip(*landmarks)
        x1, y1 = min(x_values), min(y_values)
        x2, y2 = max(x_values), max(y_values)
    else:
        # If there are not enough valid landmarks, set default values or handle as needed
        x1, y1, x2, y2 = 0, 0, 0, 0  # You can adjust these default values

    return x1, y1, x2, y2


# Loop through each JPG file and its corresponding PTS file
for jpg_path, pts_path in zip(jpg_files, pts_files):
    # Load the JPG image
    image = cv2.imread(jpg_path)

    # Parse the PTS file to get bounding box coordinates
    x1, y1, x2, y2 = generate_bbox_from_pts(pts_path)

    # Draw the bounding box on the image
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Create a segmentation mask (a white rectangle in this example)
    segmentation_mask = np.zeros_like(image)  # Initialize an all-black mask
    segmentation_mask[int(y1):int(y2), int(x1):int(x2)] = [255, 255, 255]  # Fill the bounding box with white

    # Display or save the image with the bounding box and segmentation mask
    # Display only a few of the images (e.g., display the first 3 images)
    if jpg_path in jpg_files[:3]:  # Change '3' to the number of images you want to display
        cv2.imshow('Image with Bounding Box', image)
        cv2.imshow('Segmentation Mask', segmentation_mask)
        cv2.waitKey(0)
# Close any open windows
cv2.destroyAllWindows()

# Split your dataset into training, validation, and test sets
# You can adjust the split ratios as needed
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

total_samples = len(jpg_files)
train_samples = int(train_ratio * total_samples)
val_samples = int(val_ratio * total_samples)

train_jpg_files = jpg_files[:train_samples]
val_jpg_files = jpg_files[train_samples:train_samples + val_samples]
test_jpg_files = jpg_files[train_samples + val_samples:]

# Define a custom dataset class to load and preprocess the data
class CustomDataset(Dataset):
    def _init_(self, jpg_files, pts_files, transform=None):
        self.jpg_files = jpg_files
        self.pts_files = pts_files
        self.transform = transform

    def _len_(self):
        return len(self.jpg_files)

    def _getitem_(self, idx):
        jpg_path = self.jpg_files[idx]
        pts_path = self.pts_files[idx]

        # Load the JPG image
        image = cv2.imread(jpg_path)

        # Generate bounding box coordinates from PTS file
        x1, y1, x2, y2 = generate_bbox_from_pts(pts_path)

        # Create a segmentation mask (an all-black mask in this example)
        segmentation_mask = np.zeros_like(image)

        sample = {
            'image': image,
            'bounding_box': [x1, y1, x2, y2],
            'segmentation_mask': segmentation_mask
        }

        if self.transform:
            sample = self.transform(sample)

        return sample

# Create custom datasets for training, validation, and testing
train_dataset = CustomDataset(train_jpg_files, train_jpg_files)
val_dataset = CustomDataset(val_jpg_files, val_jpg_files)
test_dataset = CustomDataset(test_jpg_files, test_jpg_files)

# Create data loaders for training, validation, and testing
batch_size = 4  # Adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define and load pre-trained Faster R-CNN model
faster_rcnn_model = fasterrcnn_resnet50_fpn(pretrained=True)
faster_rcnn_model.eval()

# Define and load pre-trained Mask R-CNN model
mask_rcnn_model = maskrcnn_resnet50_fpn(pretrained=True)
mask_rcnn_model.eval()

# Faster R-CNN Loss Function
classification_loss = nn.CrossEntropyLoss()
regression_loss = nn.SmoothL1Loss()

# Mask R-CNN Loss Function
mask_loss = nn.BCELoss()  # For binary masks, use Binary Cross-Entropy Loss

# Combine Faster R-CNN and Mask R-CNN Losses
alpha = 1.0  # Weight for Faster R-CNN classification loss
beta = 1.0   # Weight for Faster R-CNN regression loss
gamma = 1.0  # Weight for Mask R-CNN loss

# Define the combined loss
def combined_loss(classification_loss, regression_loss, mask_loss, alpha, beta, gamma):
    def loss_fn(outputs, targets):
        # Extract the outputs of both Faster R-CNN and Mask R-CNN
        faster_rcnn_outputs, mask_rcnn_outputs = outputs

        # Extract the targets for Faster R-CNN and Mask R-CNN
        faster_rcnn_targets, mask_rcnn_targets = targets

        # Compute Faster R-CNN losses
        faster_rcnn_classification_loss = classification_loss(faster_rcnn_outputs['labels'], faster_rcnn_targets['labels'])
        faster_rcnn_regression_loss = regression_loss(faster_rcnn_outputs['boxes'], faster_rcnn_targets['boxes'])

        # Compute Mask R-CNN loss
        mask_rcnn_loss = mask_loss(mask_rcnn_outputs['masks'], mask_rcnn_targets['masks'])

        # Combine the losses
        total_loss = alpha * (faster_rcnn_classification_loss + faster_rcnn_regression_loss) + gamma * mask_rcnn_loss

        return total_loss

    return loss_fn

# Create the combined loss function
combined_losses = combined_loss(classification_loss, regression_loss, mask_loss, alpha, beta, gamma)

# Set the learning rate for the optimizer
learning_rate = 0.001  # You can adjust the learning rate as needed

# Create an optimizer for the Faster R-CNN model
optimizer = Adam(faster_rcnn_model.parameters(), lr=learning_rate)

num_epochs = 10

# Training loop for Faster R-CNN
for epoch in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()  # Zero the gradients

        # Forward pass using the Faster R-CNN model
        outputs = faster_rcnn_model(batch['image'])  # Use faster_rcnn_model, not combined_model

        # Calculate the loss
        loss = combined_losses(outputs, batch['targets'])

        # Backpropagation
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Print or log training statistics, e.g., loss, accuracy

    # Validation and model evaluation
        for batch in val_loader:
            # Perform validation and evaluation

    # Save the trained Faster R-CNN model if needed
     torch.save(faster_rcnn_model.state_dict(), 'faster_rcnn_model.pth')

# Set up a separate optimizer for the Mask R-CNN model
mask_rcnn_optimizer = Adam(mask_rcnn_model.parameters(), lr=learning_rate)

# Training loop for Mask R-CNN
for epoch in range(num_epochs):
    for batch in train_loader:
        mask_rcnn_optimizer.zero_grad()  # Zero the gradients

        # Forward pass using the Mask R-CNN model
        mask_outputs = mask_rcnn_model(batch['image'])

        # Calculate the Mask R-CNN loss
        mask_loss = mask_loss(mask_outputs, batch['mask_targets'])  # Define 'mask_targets' accordingly

        # Backpropagation for Mask R-CNN
        mask_loss.backward()

        # Update Mask R-CNN model parameters
        mask_rcnn_optimizer.step()

        # Calculate the average loss for this epoch
        average_mask_loss = mask_loss / len(train_loader)

        # Print or log the training statistics for this epoch
        print(f'Epoch [{epoch+1}/{num_epochs}] - Mask Loss: {average_mask_loss}')

        # Print or log training statistics for Mask R-CNN

    # Validation and model evaluation for Mask R-CNN
    for batch in val_loader:
        # Perform validation and evaluation for Mask R-CNN

# Save the trained Mask R-CNN model if needed
torch.save(mask_rcnn_model.state_dict(), 'mask_rcnn_model.pth')

In [ ]:
#sahill

In [21]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection import maskrcnn_resnet50_fpn
import torch.nn as nn
from torch.optim import Adam


# Define paths to the dataset folders (JPG images and PTS annotations)
jpg_folder = 'C:/Users/ravis/Downloads/SHAREit/pc/file/AFW/jpg'
pts_folder = 'C:/Users/ravis/Downloads/SHAREit/pc/file/AFW/pts'

# Create lists of JPG and PTS file paths
jpg_files = [os.path.join(jpg_folder, filename) for filename in os.listdir(jpg_folder) if filename.endswith('.jpg')]
pts_files = [os.path.join(pts_folder, filename) for filename in os.listdir(pts_folder) if filename.endswith('.pts')]

# Print the list of JPG file paths
print("JPG Files:")
for jpg_file in jpg_files:
    print(jpg_file)

# Print the list of PTS file paths
print("\nPTS Files:")
for pts_file in pts_files:
    print(pts_file)

# Define a function to parse PTS annotation files and generate bounding box coordinates
def generate_bbox_from_pts(pts_file):
    with open(pts_file, 'r') as file:
        lines = file.readlines()

    # Initialize lists to store facial landmark points
    landmarks = []

    # Iterate through lines to extract facial landmark points
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 2:
            try:
                x, y = map(float, parts)
                landmarks.append((x, y))
            except ValueError:
                continue  # Skip lines that cannot be converted to float

    if len(landmarks) >= 2:
        # Calculate bounding box coordinates
        x_values, y_values = zip(*landmarks)
        x1, y1 = min(x_values), min(y_values)
        x2, y2 = max(x_values), max(y_values)
    else:
        # If there are not enough valid landmarks, set default values or handle as needed
        x1, y1, x2, y2 = 0, 0, 0, 0  # You can adjust these default values

    return x1, y1, x2, y2


# Loop through each JPG file and its corresponding PTS file
for jpg_path, pts_path in zip(jpg_files, pts_files):
    # Load the JPG image
    image = cv2.imread(jpg_path)

    # Parse the PTS file to get bounding box coordinates
    x1, y1, x2, y2 = generate_bbox_from_pts(pts_path)

    # Draw the bounding box on the image
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Create a segmentation mask (a white rectangle in this example)
    segmentation_mask = np.zeros_like(image)  # Initialize an all-black mask
    segmentation_mask[int(y1):int(y2), int(x1):int(x2)] = [255, 255, 255]  # Fill the bounding box with white

    # Display or save the image with the bounding box and segmentation mask
    # Display only a few of the images (e.g., display the first 3 images)
    if jpg_path in jpg_files[:3]:  # Change '3' to the number of images you want to display
        cv2.imshow('Image with Bounding Box', image)
        cv2.imshow('Segmentation Mask', segmentation_mask)
        cv2.waitKey(0)
# Close any open windows
cv2.destroyAllWindows()

# Split your dataset into training, validation, and test sets
# You can adjust the split ratios as needed
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

total_samples = len(jpg_files)
train_samples = int(train_ratio * total_samples)
val_samples = int(val_ratio * total_samples)

train_jpg_files = jpg_files[:train_samples]
val_jpg_files = jpg_files[train_samples:train_samples + val_samples]
test_jpg_files = jpg_files[train_samples + val_samples:]

# Define a custom dataset class to load and preprocess the data
class CustomDataset(Dataset):
    def _init_(self, jpg_files, pts_files, transform=None):
        self.jpg_files = jpg_files
        self.pts_files = pts_files
        self.transform = transform

    def _len_(self):
        return len(self.jpg_files)

    def _getitem_(self, idx):
        jpg_path = self.jpg_files[idx]
        pts_path = self.pts_files[idx]

        # Load the JPG image
        image = cv2.imread(jpg_path)

        # Generate bounding box coordinates from PTS file
        x1, y1, x2, y2 = generate_bbox_from_pts(pts_path)

        # Create a segmentation mask (an all-black mask in this example)
        segmentation_mask = np.zeros_like(image)

        sample = {
            'image': image,
            'bounding_box': [x1, y1, x2, y2],
            'segmentation_mask': segmentation_mask
        }

        if self.transform:
            sample = self.transform(sample)

        return sample

# Create custom datasets for training, validation, and testing
train_dataset = CustomDataset(train_jpg_files, train_jpg_files)
val_dataset = CustomDataset(val_jpg_files, val_jpg_files)
test_dataset = CustomDataset(test_jpg_files, test_jpg_files)

# Create data loaders for training, validation, and testing
batch_size = 4  # Adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define and load pre-trained Faster R-CNN model
faster_rcnn_model = fasterrcnn_resnet50_fpn(pretrained=True)
faster_rcnn_model.eval()

# Define and load pre-trained Mask R-CNN model
mask_rcnn_model = maskrcnn_resnet50_fpn(pretrained=True)
mask_rcnn_model.eval()

# Faster R-CNN Loss Function
classification_loss = nn.CrossEntropyLoss()
regression_loss = nn.SmoothL1Loss()

# Mask R-CNN Loss Function
mask_loss = nn.BCELoss()  # For binary masks, use Binary Cross-Entropy Loss

# Combine Faster R-CNN and Mask R-CNN Losses
alpha = 1.0  # Weight for Faster R-CNN classification loss
beta = 1.0   # Weight for Faster R-CNN regression loss
gamma = 1.0  # Weight for Mask R-CNN loss

# Define the combined loss
def combined_loss(classification_loss, regression_loss, mask_loss, alpha, beta, gamma):
    def loss_fn(outputs, targets):
        # Extract the outputs of both Faster R-CNN and Mask R-CNN
        faster_rcnn_outputs, mask_rcnn_outputs = outputs

        # Extract the targets for Faster R-CNN and Mask R-CNN
        faster_rcnn_targets, mask_rcnn_targets = targets

        # Compute Faster R-CNN losses
        faster_rcnn_classification_loss = classification_loss(faster_rcnn_outputs['labels'], faster_rcnn_targets['labels'])
        faster_rcnn_regression_loss = regression_loss(faster_rcnn_outputs['boxes'], faster_rcnn_targets['boxes'])

        # Compute Mask R-CNN loss
        mask_rcnn_loss = mask_loss(mask_rcnn_outputs['masks'], mask_rcnn_targets['masks'])

        # Combine the losses
        total_loss = alpha * (faster_rcnn_classification_loss + faster_rcnn_regression_loss) + gamma * mask_rcnn_loss

        return total_loss

    return loss_fn

# Create the combined loss function
combined_losses = combined_loss(classification_loss, regression_loss, mask_loss, alpha, beta, gamma)

# Set the learning rate for the optimizer
learning_rate = 0.001  # You can adjust the learning rate as needed

# Create an optimizer for the Faster R-CNN model
optimizer = Adam(faster_rcnn_model.parameters(), lr=learning_rate)

num_epochs = 10

# Training loop for Faster R-CNN
for epoch in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()  # Zero the gradients

        # Forward pass using the Faster R-CNN model
        outputs = faster_rcnn_model(batch['image'])  # Use faster_rcnn_model, not combined_model

        # Calculate the loss
        loss = combined_losses(outputs, batch['targets'])

        # Backpropagation
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Print or log training statistics, e.g., loss, accuracy

    # Validation and model evaluation
        for batch in val_loader:
            # Perform validation and evaluation

    # Save the trained Faster R-CNN model if needed
        torch.save(faster_rcnn_model.state_dict(), 'faster_rcnn_model.pth')


IndentationError: expected an indented block (3835902363.py, line 222)

In [ ]:

# Set up a separate optimizer for the Mask R-CNN model
mask_rcnn_optimizer = Adam(mask_rcnn_model.parameters(), lr=learning_rate)

# Training loop for Mask R-CNN
for epoch in range(num_epochs):
    for batch in train_loader:
        mask_rcnn_optimizer.zero_grad()  # Zero the gradients

        # Forward pass using the Mask R-CNN model
        mask_outputs = mask_rcnn_model(batch['image'])

        # Calculate the Mask R-CNN loss
        mask_loss = mask_loss(mask_outputs, batch['mask_targets'])  # Define 'mask_targets' accordingly

        # Backpropagation for Mask R-CNN
        mask_loss.backward()

        # Update Mask R-CNN model parameters
        mask_rcnn_optimizer.step()

        # Calculate the average loss for this epoch
        average_mask_loss = mask_loss / len(train_loader)

        # Print or log the training statistics for this epoch
        print(f'Epoch [{epoch+1}/{num_epochs}] - Mask Loss: {average_mask_loss}')

        # Print or log training statistics for Mask R-CNN

    # Validation and model evaluation for Mask R-CNN
    for batch in val_loader:
        # Perform validation and evaluation for Mask R-CNN

# Save the trained Mask R-CNN model if needed
torch.save(mask_rcnn_model.state_dict(), 'mask_rcnn_model.pth')

In [23]:
#trying to correct 
import os
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection import maskrcnn_resnet50_fpn
import torch.nn as nn
from torch.optim import Adam

# Define paths to the dataset folders (JPG images and PTS annotations)
jpg_folder = 'C:/Users/ravis/Downloads/SHAREit/pc/file/AFW/jpg'
pts_folder = 'C:/Users/ravis/Downloads/SHAREit/pc/file/AFW/pts'

# Create lists of JPG and PTS file paths
jpg_files = [os.path.join(jpg_folder, filename) for filename in os.listdir(jpg_folder) if filename.endswith('.jpg')]
pts_files = [os.path.join(pts_folder, filename) for filename in os.listdir(pts_folder) if filename.endswith('.pts')]

# Print the list of JPG file paths
print("JPG Files:")
for jpg_file in jpg_files:
    print(jpg_file)

# Print the list of PTS file paths
print("\nPTS Files:")
for pts_file in pts_files:
    print(pts_file)

# Define a function to parse PTS annotation files and generate bounding box coordinates
def generate_bbox_from_pts(pts_file):
    with open(pts_file, 'r') as file:
        lines = file.readlines()

    # Initialize lists to store facial landmark points
    landmarks = []

    # Iterate through lines to extract facial landmark points
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 2:
            try:
                x, y = map(float, parts)
                landmarks.append((x, y))
            except ValueError:
                continue  # Skip lines that cannot be converted to float

    if len(landmarks) >= 2:
        # Calculate bounding box coordinates
        x_values, y_values = zip(*landmarks)
        x1, y1 = min(x_values), min(y_values)
        x2, y2 = max(x_values), max(y_values)
    else:
        # If there are not enough valid landmarks, set default values or handle as needed
        x1, y1, x2, y2 = 0, 0, 0, 0  # You can adjust these default values

    return x1, y1, x2, y2

# Loop through each JPG file and its corresponding PTS file
for jpg_path, pts_path in zip(jpg_files, pts_files):
    # Load the JPG image
    image = cv2.imread(jpg_path)

    # Parse the PTS file to get bounding box coordinates
    x1, y1, x2, y2 = generate_bbox_from_pts(pts_path)

    # Draw the bounding box on the image
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Create a segmentation mask (a white rectangle in this example)
    segmentation_mask = np.zeros_like(image)  # Initialize an all-black mask
    segmentation_mask[int(y1):int(y2), int(x1):int(x2)] = [255, 255, 255]  # Fill the bounding box with white

    # Display or save the image with the bounding box and segmentation mask
    # Display only a few of the images (e.g., display the first 3 images)
    if jpg_path in jpg_files[:3]:  # Change '3' to the number of images you want to display
        cv2.imshow('Image with Bounding Box', image)
        cv2.imshow('Segmentation Mask', segmentation_mask)
        cv2.waitKey(0)
# Close any open windows
cv2.destroyAllWindows()

# Split your dataset into training, validation, and test sets
# You can adjust the split ratios as needed
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

total_samples = len(jpg_files)
train_samples = int(train_ratio * total_samples)
val_samples = int(val_ratio * total_samples)

train_jpg_files = jpg_files[:train_samples]
val_jpg_files = jpg_files[train_samples:train_samples + val_samples]
test_jpg_files = jpg_files[train_samples + val_samples:]

# Define a custom dataset class to load and preprocess the data
class CustomDataset(Dataset):
    def __init__(self, jpg_files, pts_files, transform=None):
        self.jpg_files = jpg_files
        self.pts_files = pts_files
        self.transform = transform

    def __len__(self):
        return len(self.jpg_files)

    def __getitem__(self, idx):
               jpg_path = self.jpg_files[idx]
        pts_path = self.pts_files[idx]

        # Load the JPG image
        image = cv2.imread(jpg_path)

        # Generate bounding box coordinates from PTS file
        x1, y1, x2, y2 = generate_bbox_from_pts(pts_path)

        # Create a segmentation mask (an all-black mask in this example)
        segmentation_mask = np.zeros_like(image)

        sample = {
            'image': image,
            'bounding_box': [x1, y1, x2, y2],
            'segmentation_mask': segmentation_mask
        }

        if self.transform:
            sample = self.transform(sample)

        return sample

# Create custom datasets for training, validation, and testing
train_dataset = CustomDataset(train_jpg_files, train_jpg_files)
val_dataset = CustomDataset(val_jpg_files, val_jpg_files)
test_dataset = CustomDataset(test_jpg_files, test_jpg_files)

# Create data loaders for training, validation, and testing
batch_size = 4  # Adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define and load pre-trained Faster R-CNN model
faster_rcnn_model = fasterrcnn_resnet50_fpn(pretrained=True)
faster_rcnn_model.eval()

# Faster R-CNN Loss Function
classification_loss = nn.CrossEntropyLoss()
regression_loss = nn.SmoothL1Loss()

# Set the learning rate for the optimizer
learning_rate = 0.001  # You can adjust the learning rate as needed

# Create an optimizer for the Faster R-CNN model
optimizer = Adam(faster_rcnn_model.parameters(), lr=learning_rate)

num_epochs = 10

# Training loop for Faster R-CNN
for epoch in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()  # Zero the gradients

        # Forward pass using the Faster R-CNN model
        outputs = faster_rcnn_model(batch['image'])  # Use faster_rcnn_model, not combined_model

        # Calculate the loss
        loss = classification_loss(outputs['labels'], batch['targets']) + regression_loss(outputs['boxes'], batch['targets'])

        # Backpropagation
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Print or log training statistics, e.g., loss, accuracy

    # Validation and model evaluation
    for batch in val_loader:
        # Perform validation and evaluation

    # Save the trained Faster R-CNN model if needed
    torch.save(faster_rcnn_model.state_dict(), 'faster_rcnn_model.pth')



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 110)